In [26]:
import requests
import numpy as np
import random

# Fetch JSON data from URL
def fetch_json(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

# Calculate the Haversine distance between two points
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Create a distance matrix from station coordinates
def create_distance_matrix(stations):
    n_stations = len(stations)
    matrix = np.zeros((n_stations, n_stations))
    for i in range(n_stations):
        for j in range(i + 1, n_stations):
            distance = haversine(stations[i]['lat'], stations[i]['lon'], stations[j]['lat'], stations[j]['lon'])
            matrix[i][j] = matrix[j][i] = distance
    return matrix

# Simulated Annealing for solving TSP
def simulated_annealing(dist_matrix, start_idx=0, temp=10000, cooling_rate=0.003, stopping_temp=1):
    n = len(dist_matrix)
    path = list(range(1, n))
    random.shuffle(path)
    path.insert(0, start_idx)
    path.append(start_idx)
    best_path = path[:]
    best_cost = path_cost(dist_matrix, path)

    while temp > stopping_temp:
        # Only shuffle between the second element and the second-to-last element
        a, b = random.sample(range(1, n), 2)
        path[a], path[b] = path[b], path[a]
        new_cost = path_cost(dist_matrix, path)
        if new_cost < best_cost or random.random() < np.exp((best_cost - new_cost) / temp):
            if new_cost < best_cost:
                best_path = path[:]
                best_cost = new_cost
        temp *= (1 - cooling_rate)

    return best_path, best_cost

# Calculate total path cost
def path_cost(matrix, path):
    return sum(matrix[path[i]][path[i+1]] for i in range(len(path) - 1))

# Main function
def main():
    url = "https://gbfs.lyft.com/gbfs/2.3/bkn/es/station_information.json"
    try:
        data = fetch_json(url)
        stations = data['data']['stations'][:20]
        print("fetching first 20 stations from the data\n")
        for i in stations:
          print(f"station {stations.index(i)} - name :{i['name']} co-ordinates :{i['lon']},{i['lat']}")
        distance_matrix = create_distance_matrix(stations)
        best_path, best_path_distance = simulated_annealing(distance_matrix)
        print("\nThe best Optimal route path:")
        for idx in best_path:
            print(f"{stations[idx]['name']} -> ", end="")
        print("\b\b ")
        print("\nTotal distance covered: {:.2f} km".format(best_path_distance))
    except requests.RequestException as e:
        print("HTTP error:", e)
    except json.JSONDecodeError:
        print("Error decoding JSON")
    except KeyError:
        print("Invalid JSON format")

if __name__ == "__main__":
    main()


fetching first 20 stations from the data

station 0 - name :6 Ave & 60 St co-ordinates :-74.013821,40.638196
station 1 - name :52 St & 6 Ave co-ordinates :-74.009441,40.642703
station 2 - name :Central Park West & W 72 St co-ordinates :-73.9762057363987,40.77579376683666
station 3 - name :55 St & 5 Ave co-ordinates :-74.013318,40.642408
station 4 - name :43 St & Skillman Ave co-ordinates :-73.9208248257637,40.74692675687384
station 5 - name :Rutland Rd & E 45 St co-ordinates :-73.93618,40.66034
station 6 - name :Warren St & W Broadway co-ordinates :-74.00910627,40.71473993
station 7 - name :Nassau Ave & Newell St co-ordinates :-73.94752621650696,40.724812564400175
station 8 - name :St Nicholas Ave & W 137 St co-ordinates :-73.947568,40.818477
station 9 - name :Broadway & W 192 St co-ordinates :-73.932173,40.857994
station 10 - name :E 18 St & Church Ave co-ordinates :-73.96316,40.64958
station 11 - name :Martense Ave & 108 St co-ordinates :-73.85418,40.74214
station 12 - name :W 54 St 